(working_with_InferenceData)=

# Working with InferenceData

Here we present a collection of common manipulations you can use while working with `InferenceData`.

In [1]:
import arviz as az
import numpy as np

In [2]:
idata = az.load_arviz_data("centered_eight")
idata

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

## Get the dataset corresponding to a single group

In [3]:
post = idata.posterior
post

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... 4.597 5.899 0.1614
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 10.59 4.523
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 8.346 7.711 5.407
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

:::{tip} 
You'll have noticed we stored the posterior group in a new variable: `post`. As `.copy()` was not called, now using `idata.posterior` or `post` is equivalent.

Use this to keep your code short yet easy to read. Store the groups you'll need very often as separate variables to use explicitly, but don't delete the InferenceData parent. You'll need it for many ArviZ functions to work properly. For example: {func}`~arviz.plot_pair` needs data from `sample_stats` group to show divergences, {func}`~arviz.compare` needs data from both `log_likelihood` and `posterior` groups, {func}`~arviz.plot_loo_pit` needs not 2 but 3 groups: `log_likelihood`, `posterior_predictive` and `posterior`.
:::

## Add a new variable


In [4]:
post["log_tau"] = np.log(post["tau"])
idata.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... 4.597 5.899 0.1614
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 10.59 4.523
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 8.346 7.711 5.407
    log_tau  (chain, draw) float64 1.316 0.7301 1.309 ... 2.122 2.043 1.688
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

## Combine chains and draws

In [5]:
stacked = az.extract_dataset(idata)
stacked

<xarray.Dataset>
Dimensions:  (school: 8, sample: 2000)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
  * sample   (sample) MultiIndex
  - chain    (sample) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3
  - draw     (sample) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
Data variables:
    mu       (sample) float64 -3.477 -2.456 -2.826 -1.996 ... 4.597 5.899 0.1614
    theta    (school, sample) float64 1.669 -6.239 2.195 ... -1.095 4.013 4.523
    tau      (sample) float64 3.73 2.075 3.703 4.146 ... 8.589 8.346 7.711 5.407
    log_tau  (sample) float64 1.316 0.7301 1.309 1.422 ... 2.122 2.043 1.688
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

You can also use {meth}`xarray.Dataset.stack` if you only want to combine the chain and draw dimensions. {func}`arviz.extract_dataset` is a convenience function aimed at taking care of the most common subsetting operations with MCMC samples. It can:
- Combine chains and draws
- Return a subset of variables (with optional filtering with regular expressions or string matching)
- Return a subset of samples. Moreover by default it returns a random subset to prevent getting non-representative samples due to bad mixing.
- Acess any group

(idata/random_subset)=
## Get a random subset of the samples

In [6]:
az.extract_dataset(idata, num_samples=100)

<xarray.Dataset>
Dimensions:  (school: 8, sample: 100)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
  * sample   (sample) MultiIndex
  - chain    (sample) int64 1 1 3 2 3 0 2 1 1 1 1 1 ... 3 0 3 1 2 1 3 3 2 1 3 2
  - draw     (sample) int64 451 357 414 113 431 260 ... 476 118 146 164 412 364
Data variables:
    mu       (sample) float64 3.494 2.461 2.294 -5.345 ... 0.2801 3.486 7.105
    theta    (school, sample) float64 11.24 8.356 2.66 ... -8.077 3.606 12.71
    tau      (sample) float64 3.404 9.254 0.9084 4.849 ... 6.928 1.582 2.167
    log_tau  (sample) float64 1.225 2.225 -0.09609 1.579 ... 1.936 0.4588 0.7734
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

:::{tip}
Use a random seed to get the same subset from multiple groups: `az.extract_dataset(idata, num_samples=100, rng=3)` and `az.extract_dataset(idata, group="log_likelihood", num_samples=100, rng=3)` will continue to have matching samples
:::

## Obtain a NumPy array for a given parameter

Let's say we want to get the values for `mu` as a NumPy array.

In [7]:
stacked.mu.values

array([-3.47698606, -2.45587061, -2.82625433, ...,  4.59705819,
        5.89850592,  0.16138927])

## Get the number of variables

Let’s check how many groups are in our hierarchical model.

In [8]:
len(idata.observed_data.school)

8

## Get coordinate values

What are the names of the groups in our hierarchical model? You can access them from the coordinate name `school` in this case

In [9]:
idata.observed_data.school

<xarray.DataArray 'school' (school: 8)>
array(['Choate', 'Deerfield', 'Phillips Andover', 'Phillips Exeter',
       'Hotchkiss', 'Lawrenceville', "St. Paul's", 'Mt. Hermon'], dtype=object)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'

## Get a subset of chains

Let’s evaluate only chain 0 and 2 here.

In [10]:
idata.sel(chain=[0, 2]).posterior

<xarray.Dataset>
Dimensions:  (chain: 2, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 2
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... -1.571 -4.435 9.763
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 12.01 16.67
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 2.812 12.18 4.453
    log_tau  (chain, draw) float64 1.316 0.7301 1.309 1.422 ... 1.034 2.5 1.494
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

## Remove the first n draws (burn-in)

Let’s say we want to remove the first 100 samples, from all the chains and all `InferenceData` groups with draws.

In [11]:
burnin = idata.sel(draw=slice(100, None))

If you check the `burnin` object you will see that the groups `posterior`, `posterior_predictive`, `prior` and `sample_stats` have 400 draws compared to `idata` that has 500. The group `observed_data` has not been affected because it does not have the `draw` dimension. Alternatively, you can specify which group or groups you want to change.

In [12]:
burnin_posterior = idata.sel(draw=slice(100, None), groups="posterior")

## Compute posterior mean values along draw and chains dimensions

If you want to compute the mean value of the posterior samples, you can simply do the following:


In [13]:
idata.posterior.mean()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    mu       float64 4.093
    theta    float64 4.56
    tau      float64 4.089
    log_tau  float64 1.15

This will effectively compute the mean along all dimensions. This is probably what you want for `mu` and `tau`, which have two dimensions (`chain` and `draw`), but maybe not what you expected for `theta`, which has one more dimension `school`. You can specify along which dimension you want to compute the mean (or other functions).

In [14]:
idata.posterior.mean(dim=['chain', 'draw'])

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 4.093
    theta    (school) float64 6.026 4.724 3.576 4.478 3.064 3.821 6.25 4.544
    tau      float64 4.089
    log_tau  float64 1.15